In [1]:
import csv
import pandas as pd
import numpy as np
import re

In [2]:
df_2021 = pd.read_csv("PERM_Disclosure_Data_FY2021.csv", low_memory=False)
df_2022 = pd.read_csv("PERM_Disclosure_Data_FY2022_Q1.csv", low_memory=False)

In [3]:
def normalize(data):
    res = data.replace(',', '')
    res = res.replace('.', '')
    res = res.upper()
    return res

def clean_up(data):
    res = data.replace('"', '')
    res = res.replace('\'', '')
    res = res.replace(',', '')
    res = res.replace('  ', '')
    res = res.upper()
    return res

In [4]:
# https://stackoverflow.com/questions/66572349/python-sub-state-names-for-abbrev-via-python-dict-with-re-sub

states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

def best_match(x):
    if len(x) == 2: # Try another way for 2-letter codes
        for a,n in states.items():
            if len(n.split()) == 2:
                if "".join([c[0] for c in n.split()]).lower() == x.lower():
                    return a.lower()
    new_rx = re.compile(r"\w*".join([ch for ch in x]), re.I)
    for a,n in states.items():
        if new_rx.match(n):
            return a.upper()

In [20]:
df_2021['EMPLOYER_NAME'] = df_2021['EMPLOYER_NAME'].apply(normalize)
df_2022['EMPLOYER_NAME'] = df_2022['EMPLOYER_NAME'].apply(normalize)
companies = pd.read_csv("companies.csv", low_memory=False)
companies

,COMPANY_ID,NAME,ADDRESS_1,ADDRESS_2,CITY,STATE,COUNTRY,ZIP_CODE
0,0,400 CAPITAL MANAGEMENT LLC,510 MADISON AVE,17TH FLOOR,NEW YORK,NY,UNITED STATES OF AMERICA,10025
1,1,ABN AMRO CLEARING CHICAGO LLC,175 W JACKSON BOULEVARD,SUITE 2050,CHICAGO,IL,UNITED STATES OF AMERICA,60604
2,2,ACHELOUS PARTNERS,800 3RD AVENUE,21ST FLOOR,NEW YORK,NY,UNITED STATES OF AMERICA,10022
3,3,AES US SERVICES LLC,4300 WILSON BLVD,NaN,ARLINGTON,VA,UNITED STATES OF AMERICA,22203
4,4,AKUNA CAPITAL LLC,333 S. WABASH AVE.,SUITE 2600,CHICAGO,IL,UNITED STATES OF AMERICA,60604
...,...,...,...,...,...,...,...,...
287,287,WEST COAST FLORIDA ENTERPRISES INC,8090 SUPPLY DRIVE,STE 100,FT. MYERS,FL,UNITED STATES OF AMERICA,33912
288,288,WILLIAM BLAIR COMPANY LLC,150 N. RIVERSIDE PLAZA,NaN,CHICAGO,IL,UNITED STATES OF AMERICA,60606
289,289,WORLDQUANT LLC,1700 EAST PUTNAM AVENUE,NaN,OLD GREENWICH,CT,UNITED STATES OF AMERICA,6870
290,290,WORLDQUANT PREDICTIVE TECHNOLOGIES,575 5TH AVE,NaN,NEW YORK CITY,NY,UNITED STATES OF AMERICA,10017


In [21]:
companies2 = pd.read_csv("companies_NSCC-MPID_fuzzymatched.csv", low_memory=False)
companies2

,COMPANY_ID,NAME,ADDRESS_1,ADDRESS_2,CITY,STATE,COUNTRY,ZIP_CODE
0,292,C6 CAPITAL SECURITIES LLC,600 Lexington Ave,32nd floor,New York,NY,UNITED STATES OF AMERICA,10022 ...
1,293,NATIONAL ALLIANCE SECURITIES CORPORATION,505 Park Ave,# 502,New York,NY,UNITED STATES OF AMERICA,10022
2,294,RIM SECURITIES LLC,400 Park Ave,NaN,New York,NY,UNITED STATES OF AMERICA,10022 ...
3,295,KMS FINANCIAL SERVICES,2001 6th Ave,#2801,Seattle,WA,UNITED STATES OF AMERICA,98121
4,296,VSR FINANCIAL SERVICES,8620 W 110TH STREET,200,OVERLAND PARK,KS,UNITED STATES OF AMERICA,66210-9651
5,297,ICP SECURITIES LLC,360 Madison Ave,NaN,New York,NY,UNITED STATES OF AMERICA,10017 ...


In [22]:
companies3 = pd.read_csv("companies_H1B_fuzzymatched.csv", low_memory=False)
companies3

,COMPANY_ID,NAME,CITY,STATE
0,298,ROSEWOOD INVESTMENT CORPORATION,NEW YORK,NY
1,299,SEMPER CAPITAL MANAGEMENT LP,NEW YORK,NY
2,300,VOYA INVESTMENT MANAGEMENT LLC,ATLANTA,GA
3,301,HQ CAPITAL PRIVATE EQUITY LLC,NEW YORK,NY
4,302,WATERFALL ASSET MANAGEMENT LLC,NEW YORK,NY
...,...,...,...,...
254,552,GATEWOOD WEALTH SOLUTIONS,ST. LOUIS,MO
255,553,B RILEY FBR INC,NEW YORK,NY
256,554,ALTISOURCE SOLUTIONS INC,PLANO,TX
257,555,PEOPLE'S UNITED ADVISORS INC,NEW YORK,NY


In [17]:
company_roles = df_2021[df_2021['EMPLOYER_NAME'].isin(companies['NAME']) | df_2021['EMPLOYER_NAME'].isin(companies2['NAME']) | df_2021['EMPLOYER_NAME'].isin(companies3['NAME'])].copy()

company_roles['YEAR'] = 0
company_roles['WORKSITE_CITY'] = company_roles['WORKSITE_CITY'].str.upper()
company_roles['WORKSITE_STATE'] = company_roles['WORKSITE_STATE'].apply(lambda x: best_match(x))

company_roles['JOB_TITLE'] = company_roles['JOB_TITLE'].str.upper()
company_roles['JOB_TITLE'] = company_roles['JOB_TITLE'].apply(clean_up)

company_roles['RECEIVED_DATE'] = pd.to_datetime(company_roles['RECEIVED_DATE'],format='%m/%d/%Y')
company_roles['DECISION_DATE'] = pd.to_datetime(company_roles['DECISION_DATE'],format='%m/%d/%Y')
company_roles['DECISION_DATE'] = pd.to_datetime(company_roles['DECISION_DATE'],format='%m/%d/%Y')

company_roles['YEAR'] = pd.DatetimeIndex(company_roles['DECISION_DATE']).year
company_roles = company_roles[['EMPLOYER_NAME',
 'YEAR',
 'WAGE_OFFER_FROM',
 'WAGE_OFFER_TO',
 'WAGE_OFFER_UNIT_OF_PAY',
 'WORKSITE_CITY',
 'WORKSITE_STATE',
 'JOB_TITLE',
 'MINIMUM_EDUCATION',
 'SPECIFIC_SKILLS'
 ]]

company_roles['WAGE_OFFER_UNIT_OF_PAY'] = 'Year'
company_roles['WAGE_OFFER_TO'].fillna(company_roles['WAGE_OFFER_FROM'], inplace=True)
company_roles['WAGE_OFFER_FROM'] = company_roles['WAGE_OFFER_FROM'].str.replace(',', '')
company_roles['WAGE_OFFER_FROM'] = company_roles['WAGE_OFFER_FROM'].str.replace('$', '')
company_roles['WAGE_OFFER_FROM'] = company_roles['WAGE_OFFER_FROM'].astype(float)
company_roles['WAGE_OFFER_FROM'] = company_roles['WAGE_OFFER_FROM'].astype(int)
company_roles['WAGE_OFFER_TO'] = company_roles['WAGE_OFFER_TO'].str.replace(',', '')
company_roles['WAGE_OFFER_TO'] = company_roles['WAGE_OFFER_TO'].str.replace('$', '')
company_roles['WAGE_OFFER_TO'] = company_roles['WAGE_OFFER_TO'].astype(float)
company_roles['WAGE_OFFER_TO'] = company_roles['WAGE_OFFER_TO'].astype(int)

cr = company_roles


<ipython-input-17-63b9713dc2ca>:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  company_roles['WAGE_OFFER_FROM'] = company_roles['WAGE_OFFER_FROM'].str.replace('$', '')
<ipython-input-17-63b9713dc2ca>:34: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  company_roles['WAGE_OFFER_TO'] = company_roles['WAGE_OFFER_TO'].str.replace('$', '')


In [18]:
company_roles2 = df_2022[df_2022['EMPLOYER_NAME'].isin(companies['NAME']) | df_2022['EMPLOYER_NAME'].isin(companies2['NAME']) | df_2022['EMPLOYER_NAME'].isin(companies3['NAME'])].copy()

company_roles2['YEAR'] = 0
company_roles2['WORKSITE_CITY'] = company_roles2['WORKSITE_CITY'].str.upper()
company_roles2['WORKSITE_STATE'] = company_roles2['WORKSITE_STATE'].apply(lambda x: best_match(x))

company_roles2['JOB_TITLE'] = company_roles2['JOB_TITLE'].str.upper()
company_roles2['JOB_TITLE'] = company_roles2['JOB_TITLE'].apply(clean_up)

company_roles2['RECEIVED_DATE'] = pd.to_datetime(company_roles2['RECEIVED_DATE'],format='%m/%d/%Y')
company_roles2['DECISION_DATE'] = pd.to_datetime(company_roles2['DECISION_DATE'],format='%m/%d/%Y')
company_roles2['DECISION_DATE'] = pd.to_datetime(company_roles2['DECISION_DATE'],format='%m/%d/%Y')

company_roles2['YEAR'] = pd.DatetimeIndex(company_roles2['DECISION_DATE']).year
company_roles2 = company_roles2[['EMPLOYER_NAME',
 'YEAR',
 'WAGE_OFFER_FROM',
 'WAGE_OFFER_TO',
 'WAGE_OFFER_UNIT_OF_PAY',
 'WORKSITE_CITY',
 'WORKSITE_STATE',
 'JOB_TITLE',
 'MINIMUM_EDUCATION',
 'SPECIFIC_SKILLS'
 ]]

company_roles2['WAGE_OFFER_UNIT_OF_PAY'] = 'Year'
company_roles2['WAGE_OFFER_TO'].fillna(company_roles2['WAGE_OFFER_FROM'], inplace=True)
company_roles2['WAGE_OFFER_FROM'] = company_roles2['WAGE_OFFER_FROM'].str.replace(',', '')
company_roles2['WAGE_OFFER_FROM'] = company_roles2['WAGE_OFFER_FROM'].str.replace('$', '')
company_roles2['WAGE_OFFER_FROM'] = company_roles2['WAGE_OFFER_FROM'].astype(float)
company_roles2['WAGE_OFFER_FROM'] = company_roles2['WAGE_OFFER_FROM'].astype(int)
company_roles2['WAGE_OFFER_TO'] = company_roles2['WAGE_OFFER_TO'].str.replace(',', '')
company_roles2['WAGE_OFFER_TO'] = company_roles2['WAGE_OFFER_TO'].str.replace('$', '')
company_roles2['WAGE_OFFER_TO'] = company_roles2['WAGE_OFFER_TO'].astype(float)
company_roles2['WAGE_OFFER_TO'] = company_roles2['WAGE_OFFER_TO'].astype(int)

cr2 = company_roles2

<ipython-input-18-f9461cb55645>:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  company_roles2['WAGE_OFFER_FROM'] = company_roles2['WAGE_OFFER_FROM'].str.replace('$', '')
<ipython-input-18-f9461cb55645>:34: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  company_roles2['WAGE_OFFER_TO'] = company_roles2['WAGE_OFFER_TO'].str.replace('$', '')


In [52]:
roles = pd.read_csv("roles.csv", low_memory=False)
company_roles = pd.read_csv("company_roles.csv", low_memory=False)
# companies = companies.append(companies2, ignore_index = True)
# companies = companies.append(companies3, ignore_index = True)
# companies = companies[['COMPANY_ID', 'NAME']]
companies

,COMPANY_ID,NAME
0,0,400 CAPITAL MANAGEMENT LLC
1,1,ABN AMRO CLEARING CHICAGO LLC
2,2,ACHELOUS PARTNERS
3,3,AES US SERVICES LLC
4,4,AKUNA CAPITAL LLC
...,...,...
552,552,GATEWOOD WEALTH SOLUTIONS
553,553,B RILEY FBR INC
554,554,ALTISOURCE SOLUTIONS INC
555,555,PEOPLE'S UNITED ADVISORS INC


In [53]:
company_roles = company_roles.merge(companies, on='COMPANY_ID', how='left')
company_roles = company_roles.merge(roles, on='ROLE_ID', how='left')
company_roles

,COMPANY_ROLES_ID,COMPANY_ID,ROLE_ID,NAME_x,NAME_y
0,0,0,3763,400 CAPITAL MANAGEMENT LLC,ASSOCIATE QUANTITATIVE ANALYTICS
1,1,0,11079,400 CAPITAL MANAGEMENT LLC,INVESTMENT ANALYST
2,2,0,588,400 CAPITAL MANAGEMENT LLC,ANALYST FINANCE
3,3,1,9307,ABN AMRO CLEARING CHICAGO LLC,EXCHANGE FEE CONTROL ACCOUNTANT
4,4,1,19469,ABN AMRO CLEARING CHICAGO LLC,STOCK LOAN OPERATIONS ASSOCIATE
...,...,...,...,...,...
31680,31680,506,9043,THE HUNTINGTON NATIONAL BANK,ENGINEER
31681,31681,506,11962,THE HUNTINGTON NATIONAL BANK,LEAD ENGINEER
31682,31682,506,14776,THE HUNTINGTON NATIONAL BANK,QUANTITATIVE ANALYST
31683,31683,506,19732,THE HUNTINGTON NATIONAL BANK,SYSTEMS & APP TECHNOLOGIST


In [54]:
company_roles.rename(columns={'NAME_x': 'COMPANY_NAME', 'NAME_y': 'ROLE_NAME'}, inplace=True)
company_roles

,COMPANY_ROLES_ID,COMPANY_ID,ROLE_ID,COMPANY_NAME,ROLE_NAME
0,0,0,3763,400 CAPITAL MANAGEMENT LLC,ASSOCIATE QUANTITATIVE ANALYTICS
1,1,0,11079,400 CAPITAL MANAGEMENT LLC,INVESTMENT ANALYST
2,2,0,588,400 CAPITAL MANAGEMENT LLC,ANALYST FINANCE
3,3,1,9307,ABN AMRO CLEARING CHICAGO LLC,EXCHANGE FEE CONTROL ACCOUNTANT
4,4,1,19469,ABN AMRO CLEARING CHICAGO LLC,STOCK LOAN OPERATIONS ASSOCIATE
...,...,...,...,...,...
31680,31680,506,9043,THE HUNTINGTON NATIONAL BANK,ENGINEER
31681,31681,506,11962,THE HUNTINGTON NATIONAL BANK,LEAD ENGINEER
31682,31682,506,14776,THE HUNTINGTON NATIONAL BANK,QUANTITATIVE ANALYST
31683,31683,506,19732,THE HUNTINGTON NATIONAL BANK,SYSTEMS & APP TECHNOLOGIST


In [42]:
cr.reset_index(drop=True, inplace=True)
cr.drop_duplicates(inplace=True)

In [43]:
cr.sort_values(['EMPLOYER_NAME'], inplace=True)

In [44]:
cr = cr.reset_index(drop=True)

In [45]:
cr2.sort_values(['EMPLOYER_NAME'], inplace=True)
cr = cr.append(cr2, ignore_index=True)
cr = cr.reset_index(drop=True)
cr

,EMPLOYER_NAME,YEAR,WAGE_OFFER_FROM,WAGE_OFFER_TO,WAGE_OFFER_UNIT_OF_PAY,WORKSITE_CITY,WORKSITE_STATE,JOB_TITLE,MINIMUM_EDUCATION,SPECIFIC_SKILLS
0,400 CAPITAL MANAGEMENT LLC,2020,150000,150000,Year,NEW YORK,NY,ASSOCIATE DIRECTOR QUANTITATIVE RESEARCH,NaN,NaN
1,ABN AMRO CLEARING CHICAGO LLC,2021,150000,150000,Year,CHICAGO,IL,VICE PRESIDENT BUSINESS DEVELOPMENT ANALYST,NaN,NaN
2,ACADIAN ASSET MANAGEMENT LLC,2021,87500,87500,Year,BOSTON,MA,PERFORMANCE & ATTRIBUTION ANALYST,NaN,NaN
3,ACADIAN ASSET MANAGEMENT LLC,2021,140000,150000,Year,BOSTON,MA,SENIOR INFRASTRUCTURE ENGINEER,NaN,NaN
4,ACADIAN ASSET MANAGEMENT LLC,2021,132500,142500,Year,BOSTON,MA,SENIOR SOFTWARE ENGINEER,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
6981,WELLS FARGO BANK NA,2021,171300,188500,Year,MINNEAPOLIS,MN,EXTERNAL FINANCIAL REPORTING LEAD CONSULTANT,Bachelor's,Position requires a Bachelors degree in Busine...
6982,WELLS FARGO BANK NA,2021,122616,167700,Year,CHARLOTTE,NC,APPLICATION SYSTEMS ENGINEER 5,Bachelor's,Position requires a Bachelors degree in Comput...
6983,WILLIAM BLAIR COMPANY LLC,2021,225000,225000,Year,CHICAGO,IL,VICE PRESIDENT,Bachelor's,Education Requirement Bachelors degree or fore...
6984,WORLDQUANT LLC,2021,100000,130000,Year,NEW YORK,NY,ASSOCIATE QUANTITATIVE RESEARCH & OPTIMIZATION,Master's,"Background in education, training or experienc..."


In [31]:
cr.to_csv('temp_specs.csv')

In [32]:
cr = cr.drop('MINIMUM_EDUCATION', 1)
cr = cr.drop('SPECIFIC_SKILLS', 1)

<ipython-input-32-656b3419ad7e>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  cr = cr.drop('MINIMUM_EDUCATION', 1)
<ipython-input-32-656b3419ad7e>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  cr = cr.drop('SPECIFIC_SKILLS', 1)


In [33]:
cr.drop_duplicates(ignore_index=True)
cr

,EMPLOYER_NAME,YEAR,WAGE_OFFER_FROM,WAGE_OFFER_TO,WAGE_OFFER_UNIT_OF_PAY,WORKSITE_CITY,WORKSITE_STATE,JOB_TITLE
0,400 CAPITAL MANAGEMENT LLC,2020,150000,150000,Year,NEW YORK,NY,ASSOCIATE DIRECTOR QUANTITATIVE RESEARCH
1,ABN AMRO CLEARING CHICAGO LLC,2021,150000,150000,Year,CHICAGO,IL,VICE PRESIDENT BUSINESS DEVELOPMENT ANALYST
2,ACADIAN ASSET MANAGEMENT LLC,2021,98000,108000,Year,BOSTON,MA,DEVOPS ENGINEER
3,ACADIAN ASSET MANAGEMENT LLC,2021,148000,158000,Year,BOSTON,MA,SENIOR BUSINESS ANALYST
4,ACADIAN ASSET MANAGEMENT LLC,2021,162000,172000,Year,BOSTON,MA,SENIOR SOFTWARE ENGINEER
...,...,...,...,...,...,...,...,...
6191,WELLS FARGO BANK NA,2021,122741,167700,Year,MINNEAPOLIS,MN,APPLICATION SYSTEMS ENGINEER 5
6192,WELLS FARGO BANK NA,2021,150200,201500,Year,CHARLOTTE,NC,SOFTWARE ENGINEER SR.
6193,WILLIAM BLAIR COMPANY LLC,2021,225000,225000,Year,CHICAGO,IL,VICE PRESIDENT
6194,WORLDQUANT LLC,2021,100000,130000,Year,NEW YORK,NY,ASSOCIATE QUANTITATIVE RESEARCH & OPTIMIZATION


In [34]:
cr_temp = cr.groupby(['EMPLOYER_NAME', 'YEAR', 'WORKSITE_CITY', 'WORKSITE_STATE', 'JOB_TITLE']).agg({'WAGE_OFFER_FROM' : ['min'], 'WAGE_OFFER_TO' : ['max']})
cr_temp

WAGE_OFFER_FROM  \
                                                                                                                                    min   
EMPLOYER_NAME                      YEAR WORKSITE_CITY WORKSITE_STATE JOB_TITLE                                                            
400 CAPITAL MANAGEMENT LLC         2020 NEW YORK      NY             ASSOCIATE DIRECTOR QUANTITATIVE RESEARCH                    150000   
ABN AMRO CLEARING CHICAGO LLC      2021 CHICAGO       IL             VICE PRESIDENT BUSINESS DEVELOPMENT ANALYST                 150000   
ACADIAN ASSET MANAGEMENT LLC       2020 BOSTON        MA             ANALYST IMPLEMENTATION                                       96000   
                                   2021 BOSTON        MA             DEVOPS ENGINEER                                              98000   
                                                                     PERFORMANCE & ATTRIBUTION ANALYST                            87500   
...                                                                                                                                 ...   
WORLDQUANT LLC                     2021 OLD GREENWICH CT             BUSINESS DEVELOPMENT ASSOCIATE                              101000   
                                                                     CHIEF STRATEGY OFFICER                                      140000   
                                                                     VICE PRESIDENT PORTFOLIO MANAGEMENT (WITH SPECI...          100000   
WORLDQUANT PREDICTIVE TECHNOLOGIES 2021 NEW YORK      NY             SENIOR RESEARCHER                                           130000   
XCHANGE FINANCIAL ACCESS LLC       2021 CHICAGO       IL             QUANTITATIVE DEVELOPER                                       72000   

                                                                                                                        WAGE_OFFER_TO  
                                                                                                                                  max  
EMPLOYER_NAME                      YEAR WORKSITE_CITY WORKSITE_STATE JOB_TITLE                                                         
400 CAPITAL MANAGEMENT LLC         2020 NEW YORK      NY             ASSOCIATE DIRECTOR QUANTITATIVE RESEARCH                  150000  
ABN AMRO CLEARING CHICAGO LLC      2021 CHICAGO       IL             VICE PRESIDENT BUSINESS DEVELOPMENT ANALYST               150000  
ACADIAN ASSET MANAGEMENT LLC       2020 BOSTON        MA             ANALYST IMPLEMENTATION                                    116000  
                                   2021 BOSTON        MA             DEVOPS ENGINEER                                           108000  
                                                                     PERFORMANCE & ATTRIBUTION ANALYST                          87500  
...                                                                                                                               ...  
WORLDQUANT LLC                     2021 OLD GREENWICH CT             BUSINESS DEVELOPMENT ASSOCIATE                            131000  
                                                                     CHIEF STRATEGY OFFICER                                    170000  
                                                                     VICE PRESIDENT PORTFOLIO MANAGEMENT (WITH SPECI...        130000  
WORLDQUANT PREDICTIVE TECHNOLOGIES 2021 NEW YORK      NY             SENIOR RESEARCHER                                         170000  
XCHANGE FINANCIAL ACCESS LLC       2021 CHICAGO       IL             QUANTITATIVE DEVELOPER                                     72000  

[4417 rows x 2 columns]

In [35]:
cr_temp.to_csv('cr_temp.csv')

In [55]:
cr_final = pd.read_csv('cr_temp.csv', low_memory=False)
cr_final

,EMPLOYER_NAME,YEAR,WORKSITE_CITY,WORKSITE_STATE,JOB_TITLE,WAGE_OFFER_FROM,WAGE_OFFER_TO
0,400 CAPITAL MANAGEMENT LLC,2020,NEW YORK,NY,ASSOCIATE DIRECTOR QUANTITATIVE RESEARCH,150000,150000
1,ABN AMRO CLEARING CHICAGO LLC,2021,CHICAGO,IL,VICE PRESIDENT BUSINESS DEVELOPMENT ANALYST,150000,150000
2,ACADIAN ASSET MANAGEMENT LLC,2020,BOSTON,MA,ANALYST IMPLEMENTATION,96000,116000
3,ACADIAN ASSET MANAGEMENT LLC,2021,BOSTON,MA,DEVOPS ENGINEER,98000,108000
4,ACADIAN ASSET MANAGEMENT LLC,2021,BOSTON,MA,PERFORMANCE & ATTRIBUTION ANALYST,87500,87500
...,...,...,...,...,...,...,...
4412,WORLDQUANT LLC,2021,OLD GREENWICH,CT,BUSINESS DEVELOPMENT ASSOCIATE,101000,131000
4413,WORLDQUANT LLC,2021,OLD GREENWICH,CT,CHIEF STRATEGY OFFICER,140000,170000
4414,WORLDQUANT LLC,2021,OLD GREENWICH,CT,VICE PRESIDENT PORTFOLIO MANAGEMENT (WITH SPEC...,100000,130000
4415,WORLDQUANT PREDICTIVE TECHNOLOGIES,2021,NEW YORK,NY,SENIOR RESEARCHER,130000,170000


In [56]:
cr_final.rename(columns={'EMPLOYER_NAME': 'COMPANY_NAME', 'JOB_TITLE': 'ROLE_NAME'}, inplace=True)
crs = pd.merge(company_roles, cr_final,  how='right', on=['COMPANY_NAME','ROLE_NAME'])
crs

,COMPANY_ROLES_ID,COMPANY_ID,ROLE_ID,COMPANY_NAME,ROLE_NAME,YEAR,WORKSITE_CITY,WORKSITE_STATE,WAGE_OFFER_FROM,WAGE_OFFER_TO
0,29375,0,2989,400 CAPITAL MANAGEMENT LLC,ASSOCIATE DIRECTOR QUANTITATIVE RESEARCH,2020,NEW YORK,NY,150000,150000
1,14,1,20979,ABN AMRO CLEARING CHICAGO LLC,VICE PRESIDENT BUSINESS DEVELOPMENT ANALYST,2021,CHICAGO,IL,150000,150000
2,20651,481,655,ACADIAN ASSET MANAGEMENT LLC,ANALYST IMPLEMENTATION,2020,BOSTON,MA,96000,116000
3,20674,481,7879,ACADIAN ASSET MANAGEMENT LLC,DEVOPS ENGINEER,2021,BOSTON,MA,98000,108000
4,20664,481,13674,ACADIAN ASSET MANAGEMENT LLC,PERFORMANCE & ATTRIBUTION ANALYST,2021,BOSTON,MA,87500,87500
...,...,...,...,...,...,...,...,...,...,...
4414,19185,289,5870,WORLDQUANT LLC,BUSINESS DEVELOPMENT ASSOCIATE,2021,OLD GREENWICH,CT,101000,131000
4415,31225,289,6361,WORLDQUANT LLC,CHIEF STRATEGY OFFICER,2021,OLD GREENWICH,CT,140000,170000
4416,31223,289,21653,WORLDQUANT LLC,VICE PRESIDENT PORTFOLIO MANAGEMENT (WITH SPEC...,2021,OLD GREENWICH,CT,100000,130000
4417,19221,290,17480,WORLDQUANT PREDICTIVE TECHNOLOGIES,SENIOR RESEARCHER,2021,NEW YORK,NY,130000,170000


In [57]:
crs = crs.drop(['COMPANY_ID', 'ROLE_ID', 'COMPANY_NAME','ROLE_NAME'], 1)
crs = crs.rename(columns={'WORKSITE_CITY': 'CITY', 'WORKSITE_STATE': 'STATE', 'WAGE_OFFER_FROM': 'MIN_SALARY', 'WAGE_OFFER_TO': 'MAX_SALARY'})
crs

<ipython-input-57-cf4bf7b55f71>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  crs = crs.drop(['COMPANY_ID', 'ROLE_ID', 'COMPANY_NAME','ROLE_NAME'], 1)


,COMPANY_ROLES_ID,YEAR,CITY,STATE,MIN_SALARY,MAX_SALARY
0,29375,2020,NEW YORK,NY,150000,150000
1,14,2021,CHICAGO,IL,150000,150000
2,20651,2020,BOSTON,MA,96000,116000
3,20674,2021,BOSTON,MA,98000,108000
4,20664,2021,BOSTON,MA,87500,87500
...,...,...,...,...,...,...
4414,19185,2021,OLD GREENWICH,CT,101000,131000
4415,31225,2021,OLD GREENWICH,CT,140000,170000
4416,31223,2021,OLD GREENWICH,CT,100000,130000
4417,19221,2021,NEW YORK,NY,130000,170000


In [58]:
crs = crs[['COMPANY_ROLES_ID', 'YEAR', 'MIN_SALARY', 'MAX_SALARY', 'CITY', 'STATE']]
crs

,COMPANY_ROLES_ID,YEAR,MIN_SALARY,MAX_SALARY,CITY,STATE
0,29375,2020,150000,150000,NEW YORK,NY
1,14,2021,150000,150000,CHICAGO,IL
2,20651,2020,96000,116000,BOSTON,MA
3,20674,2021,98000,108000,BOSTON,MA
4,20664,2021,87500,87500,BOSTON,MA
...,...,...,...,...,...,...
4414,19185,2021,101000,131000,OLD GREENWICH,CT
4415,31225,2021,140000,170000,OLD GREENWICH,CT
4416,31223,2021,100000,130000,OLD GREENWICH,CT
4417,19221,2021,130000,170000,NEW YORK,NY


In [63]:
crs2 = pd.read_csv('company_role_specs_updated.csv')
crs2 = crs2.drop(['COMPANY_ROLES_SPECS_ID'], 1)
crs2 = crs2.append(crs, ignore_index=True)
crs2

<ipython-input-63-03d3aefbf8ff>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  crs2 = crs2.drop(['COMPANY_ROLES_SPECS_ID'], 1)


,COMPANY_ROLES_ID,YEAR,MIN_SALARY,MAX_SALARY,CITY,STATE
0,0,2016,125000,125000,NEW YORK,NY
1,0,2017,125000,125000,NEW YORK,NY
2,0,2018,106371,106371,NEW YORK,NY
3,1,2019,71920,71920,NEW YORK,NY
4,0,2019,106371,106371,NEW YORK,NY
...,...,...,...,...,...,...
68774,19185,2021,101000,131000,OLD GREENWICH,CT
68775,31225,2021,140000,170000,OLD GREENWICH,CT
68776,31223,2021,100000,130000,OLD GREENWICH,CT
68777,19221,2021,130000,170000,NEW YORK,NY


In [68]:
cr_temp = crs2.groupby(['COMPANY_ROLES_ID', 'YEAR', 'CITY', 'STATE']).agg({'MIN_SALARY' : ['min'], 'MAX_SALARY' : ['max']})
cr_temp.to_csv('crs_temp.csv')


In [69]:
company_role_specs = pd.read_csv('crs_temp.csv')
company_role_specs = company_role_specs[['COMPANY_ROLES_ID', 'YEAR', 'MIN_SALARY', 'MAX_SALARY', 'CITY', 'STATE']]
company_role_specs.index.name = 'COMPANY_ROLE_SPECS_ID'
company_role_specs.to_csv('company_role_specs.csv')